# XRD Data Analysis for perovskite (MHP)

This Python script allows the analysis of X-ray Diffraction (XRD) data from .csv files containing measurements with 2θ and intensity columns. It provides several useful features for processing XRD spectra.

---

## 📌 Features

Import .csv files containing multiple XRD measurements

Peak features extracting :

**Integration of the area** under the curve between specific bounds

The **maximum intensity** of the peaks

The **peak positions** in 2θ

The **FWHM** (Full Width at Half Maximum)


## 🔧 Usage

Load a .csv file containing 2θ and intensity columns.

Define integration boundaries.

Retrieve computed values.

---
Navid MOUHAMAD, XRD Data Analysis for perovskite (MHP). Available at: https://github.com/Nav9110/XRD_code


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.interpolate import interp1d
from numpy import trapz
from scipy.signal import find_peaks, peak_widths
from scipy.interpolate import UnivariateSpline

In [ ]:
# Chemin du répertoire + nom du fichier 
chemin = r'C:\Users\navid.mouhamad\OneDrive - IPVF\Desktop\Stages 2025\1ere_campagne\XRD\XRD_campagne_1.csv'

# Séparateurs possible en .csv : "," ou ";"
df = pd.read_csv(chemin, sep=';')
df.head(3)

In [ ]:
def index_to_xdata(xdata, indices):
    "interpolate the values from signal.peak_widths to xdata"
    ind = np.arange(len(xdata))
    f = interp1d(ind,xdata)
    return f(indices)

In [ ]:
# Paramètres
spline_smoothness = 100000 # pour le fit de pic
peak_height = 120 
peak_prominence = 100 

# Bornes manuelles d'intégration
borne_PbI2 = (11.8, 12.8)
borne_PSK = (13.0, 14.2)
default_pos_PbI2 = 12.25  # Position par défaut si PbI2 non détecté

# Listes de stockage
area_totale = []
area_PbI2, position_PbI2, fwhm_PbI2, max_pic_PbI2 = [], [], [], []
area_PSK, position_PSK, fwhm_PSK, max_pic_PSK = [], [], [], []

# Boucle sur les colonnes
for i in range(0, len(df.columns), 2):
    dataset = i
    if dataset >= len(df.columns):
        print("Index hors limite :", dataset)
        continue

    # Données brutes
    x = df.iloc[:, dataset]
    y = df.iloc[:, dataset + 1]
    peaks_name = ['PbI2', 'PSK']

    # Lissage
    x_new = np.linspace(x.min(), x.max(), 1000)
    spline = UnivariateSpline(x, y, s=spline_smoothness)
    y_smooth = spline(x_new)

    # Interpolation
    interpolation = interp1d(x_new, y_smooth, kind='linear', fill_value='extrapolate')
    y_inter = interpolation(x_new)

    # Intégrale totale
    aire_totale = trapz(y_inter, x_new)
    area_totale.append(round(aire_totale, 2))
    print(f"Échantillon : {df.columns[dataset]}")
    print(f"Intégrale totale : {round(aire_totale, 2)} a.u.°")

    # Détection des pics
    peaks, props = find_peaks(y_smooth, height=peak_height, prominence=peak_prominence)

    # Gestion manuelle si un seul pic trouvé (pas de PbI2 formé)
    if len(peaks) < 2:
        print(f"PbI2 non détecté, position par défaut utilisée : {default_pos_PbI2}")

        # PSK détecté
        pos_PSK = x_new[peaks[0]]
        int_PSK = y_smooth[peaks[0]]
        position_PSK.append(round(pos_PSK, 2))
        max_pic_PSK.append(round(int_PSK, 2))

        # FWHM uniquement pour PSK
        widths, height_fwhm, left_ips, right_ips = peak_widths(y_smooth, peaks, rel_height=0.5)
        fwhm_PSK_val = np.interp(right_ips, np.arange(len(x_new)), x_new) - np.interp(left_ips, np.arange(len(x_new)), x_new)
        fwhm_PSK.append(round(fwhm_PSK_val[0], 2))
        fwhm_PbI2.append(0)
        position_PbI2.append(default_pos_PbI2)
        max_pic_PbI2.append(0)

    else:
        # PbI2 et PSK détectés
        pos_peaks = x_new[peaks]
        int_peaks = y_smooth[peaks]

        position_PbI2.append(round(pos_peaks[0], 2))
        max_pic_PbI2.append(round(int_peaks[0], 2))
        position_PSK.append(round(pos_peaks[1], 2))
        max_pic_PSK.append(round(int_peaks[1], 2))

        widths, height_fwhm, left_ips, right_ips = peak_widths(y_smooth, peaks, rel_height=0.5)
        fwhm_vals = np.interp(right_ips, np.arange(len(x_new)), x_new) - np.interp(left_ips, np.arange(len(x_new)), x_new)

        fwhm_PbI2.append(round(fwhm_vals[0], 2))
        fwhm_PSK.append(round(fwhm_vals[1], 2))

        print(f"PbI2 : {round(pos_peaks[0],2)}° | {round(int_peaks[0],2)} a.u. | FWHM: {round(fwhm_vals[0],2)}°")
        print(f"PSK  : {round(pos_peaks[1],2)}° | {round(int_peaks[1],2)} a.u. | FWHM: {round(fwhm_vals[1],2)}°")

    # Intégration manuelle : PbI2
    x_pb = np.linspace(borne_PbI2[0], borne_PbI2[1], 1000)
    y_pb = interpolation(x_pb)
    aire_pb = trapz(y_pb, x_pb)
    area_PbI2.append(round(aire_pb, 2))
    print(f"Aire PbI2 : {round(aire_pb, 2)} a.u.°")

    # Intégration manuelle : PSK
    x_psk = np.linspace(borne_PSK[0], borne_PSK[1], 1000)
    y_psk = interpolation(x_psk)
    aire_psk = trapz(y_psk, x_psk)
    area_PSK.append(round(aire_psk, 2))
    print(f"Aire PSK : {round(aire_psk, 2)} a.u.°")

    # Affichage
    plt.figure(figsize=(8, 5))
    plt.plot(x, y, '-', label='Données brutes')
    plt.plot(x_new, y_smooth, '-', label='Spline Fit')
    plt.plot(x_pb, y_pb, '--', label='Intégrale PbI2')
    plt.plot(x_psk, y_psk, '--', label='Intégrale PSK')
    plt.fill_between(x_pb, y_pb, alpha=0.3)
    plt.fill_between(x_psk, y_psk, alpha=0.3)

    if len(peaks) > 0:
        plt.plot(x_new[peaks], y_smooth[peaks], 'x', label='Pics détectés')
        plt.hlines(height_fwhm,
                   np.interp(left_ips, np.arange(len(x_new)), x_new),
                   np.interp(right_ips, np.arange(len(x_new)), x_new),
                   color='r', label='FWHM')

    plt.title(f"Analyse : {df.columns[dataset]}")
    plt.xlabel("2θ (°)")
    plt.ylabel("Intensité (a.u.)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# Calcul des % : (PbI2*100)/(PSK+PbI2) 
percent_integrale = (np.array(area_PbI2) * 100) / (np.array(area_PbI2) + np.array(area_PSK))
percent_int_pic = (np.array(max_pic_PbI2) * 100) / (np.array(max_pic_PbI2) + np.array(max_pic_PSK))
percent_fwhm = (np.array(fwhm_PbI2) * 100) / (np.array(fwhm_PbI2) + np.array(fwhm_PSK))
percent_integrale = [round(i, 2) for i in percent_integrale]
percent_int_pic = [round(i, 2) for i in percent_int_pic]
percent_fwhm = [round(i, 2) for i in percent_fwhm]

# Calcul des ratios : PbI2/PSK 
ratio_integrale = (np.array(area_PbI2)) / (np.array(area_PSK))
ratio_int_pic = (np.array(max_pic_PbI2)) / (np.array(max_pic_PSK))
ratio_fwhm = (np.array(fwhm_PbI2)) / (np.array(fwhm_PSK))
ratio_integrale = [round(i, 2) for i in ratio_integrale]
ratio_int_pic = [round(i, 2) for i in ratio_int_pic]
ratio_fwhm = [round(i, 2) for i in ratio_fwhm]

# Création du DataFrame final
sample_names = df.columns[::2].values  # Prendre les colonnes impaires (noms des échantillons)

# Créer le DataFrame avec les noms des échantillons et les autres données calculées
XRD_treated = pd.DataFrame({
    "Nom échantillon": sample_names,
    "Aire totale": area_totale,
    "Aire PbI2": area_PbI2,
    "Aire PSK": area_PSK,
    "Position PbI2": position_PbI2,
    "Position PSK": position_PSK,
    "FWHM PbI2": fwhm_PbI2,
    "FWHM PSK": fwhm_PSK,
    "Max PbI2": max_pic_PbI2,
    "Max PSK": max_pic_PSK,
    "% PbI2 (Intégrale)": percent_integrale,
    "% PbI2 (Pic)": percent_int_pic,
    "% PbI2 (FWHM)": percent_fwhm,
    "Ratio (Intégrale)": ratio_integrale,
    "Ratio (Pic)": ratio_int_pic,
    "Ratio (FWHM)": ratio_fwhm
})

# Export en CSV, dans le dossier ou se trouve le code
XRD_treated.to_csv("XRD_treated.csv", index=False)

XRD_treated.head(2)
